In [ ]:
from requests import request
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd

# === Generic setup: local variables and helpers ===

# The URL of the server serving the WebDP API
server_api_root = "http://localhost:8080/v1"

# The root user token is used to bootstrap the server and create new users
# (But it can also run queries if you don't care about users and roles.)
rootToken = str(input("rootToken: "))

# A helper for making HTTP requests
def make_request(endpoint, method, token=None, body=None, content_type="application/json"):
    headers = { "Content-Type": content_type }
    if token is not None:
        headers["Authorization"] = f"Bearer {token}"
    request_args={
      "url": f"{server_api_root}{endpoint}",
      "method": method,
      "headers": headers,
    }
    if content_type == "application/json" and body is not None:
      request_args["json"] = body
    if content_type == "text/csv" and body is not None:
      request_args["data"] = body

    return request(**request_args)

In [ ]:
# === Admin ===

# As the root user (or any user with admin privileges), one can create different
# users and assign them some user roles. In this case, we will create a data
# curator + admin and two data analysts
users = [
    { "name": "John", "handle": "john", "password": "s09sf&n1", "roles": ["Admin", "Curator"] },
    { "name": "Jane", "handle": "jane", "password": "fa098@$s", "roles": ["Analyst"] },
    { "name": "Java", "handle": "java", "password": "*S89dj21", "roles": ["Analyst"] },
]
for user in users:
    response = make_request(endpoint="/users", method="POST", token=rootToken, body=user)
    pprint(response)

In [ ]:
# === Curator ===

# Now, let's pretend we are John (Curator) and we login to the server
response = make_request(endpoint="/login", method="POST", body={"username": "john", "password": "s09sf&n1"})
pprint(response)
pprint(response.json())
johnToken = response.json()["jwt"]

# As a Curator we can create create a new dataset
response = make_request(endpoint="/datasets", method="POST", token=johnToken, body={
    "name": "salaries",
    "owner": "john",
    "schema": [
        { "name": "name",   "type": { "name": "Text" } },
        { "name": "age",    "type": { "name": "Int", "low": 18, "high": 100 } },
        { "name": "job",    "type": { "name": "Enum", "labels": ["Accountant", "Dentist", "High School Teacher", "Software Engineer"] } },
        { "name": "salary", "type": { "name": "Int", "low": 0, "high": 100000 } }
    ],
    "privacy_notion": "PureDP",
    "total_budget": { "epsilon": 5 }
})
pprint(response)
pprint(response.json())
datasetId = response.json()["id"]

# We can check if this new dataset exists in the system
response = make_request(endpoint="/datasets", method="GET", token=johnToken)
pprint(response)
pprint(response.json())

# We can load the dataset
with open("salaries.csv") as csv:
    data = csv.read()

response = make_request(
    endpoint=f"/dataset/{datasetId}/upload",
    method="POST",
    token=johnToken,
    content_type="text/csv",
    body=data.encode()
)
pprint(response)

# And we can distribute some of our dataset's budget across users
budget_allocation = [
    ("john", { "epsilon": 2.5 }),
    ("jane", { "epsilon": 1 }),
    ("java", { "epsilon": 0.5 })
]

for (user, budget) in budget_allocation:
    response = make_request(
        endpoint=f"/budget/allocation/{user}/{datasetId}",
        method="POST",
        token=johnToken,
        body=budget
    )
    pprint(response)

# At any point we can check the current budget allocation for our datasets
response = make_request(endpoint=f"/budget/dataset/{datasetId}", method="GET", token=johnToken)
pprint(response)
pprint(response.json())

In [ ]:
# === Analyst ===

# As Curator, John's job here is done, we can move on into the data analyst world now
response = make_request(endpoint="/login", method="POST", body={"username": "jane", "password": "fa098@$s"})
pprint(response)
pprint(response.json())
janeToken = response.json()["jwt"]

# As analyst, we can check our allocated budget
response = make_request(endpoint=f"/budget/user/jane", method="GET", token=janeToken)
pprint(response)
pprint(response.json())

# If we have available budget we can execute some queries, for instance:

# Query #1: count the number of rows in the dataset
response = make_request(
    endpoint=f"/query/evaluate",
    method="POST",
    token=janeToken,
    body={
        "dataset": datasetId,
        "budget": { "epsilon": 0.2 },
        "query": [
            { "count": {} }
        ]
    }
)
pprint(response)
# We can extract the result from the response's JSON object
pprint(response.json()["rows"])

# Query #2: get the average salary per profession
response = make_request(
    endpoint=f"/query/evaluate",
    method="POST",
    token=janeToken,
    body={
        "dataset": datasetId,
        "budget": { "epsilon": 0.5 },
        "query": [
            { "groupby": { "job": ["Accountant", "Dentist", "High School Teacher", "Software Engineer"] } },
            { "mean": { "column": "salary" } }
        ]
    }
)
pprint(response)
# We could use pandas to visualize the result as a bar chart
pd.DataFrame(response.json()["rows"]).plot.bar(x="job", y="mean_salary")

# Query #3: try get the mean salary of young software engineers
response = make_request(
    endpoint=f"/query/evaluate",
    method="POST",
    token=janeToken,
    body={
        "dataset": datasetId,
        "budget": { "epsilon": 0.5 }, # Exceeding our remaining budget
        "query": [
            { "filter": [ "age > 18", "age < 35", 'job == "Software Engineer"' ] },
            { "mean": { "column": "salary" } }
        ]
    }
)
pprint(response)

# This request should prompt a 400 error with message:
# "The user's available budget is not sufficient to run this query"
pprint(response.json())